In [39]:
import nltk
#nltk.download('punkt')
#!pip install seqeval
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import gensim
from gensim.models import Word2Vec

In [40]:
df=open('/content/drive/My Drive/knen/data/rawtext.txt',encoding='utf-8').read().split('\n')
data=[]
for i in df:
    temp=[]
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [41]:
#using skip gram
w2v = gensim.models.Word2Vec(data, min_count = 1, size = 200, window = 5, sg = 1) 
print("'likes' " + "and 'views' - Skip Gram : ", w2v.similarity('likes', 'views'))

'likes' and 'views' - Skip Gram :  0.8728599


In [42]:
vocab=list(w2v.wv.vocab)

chars=[]
for word in vocab:
    temp=[]
    for char in word:
        temp.append(char)
    chars.append(temp)

c2v = gensim.models.Word2Vec(chars, min_count = 1, size = 30, window = 5, sg = 1) 


In [43]:
#sub word embedding copy
#!pip install BPEmb
from bpemb import BPEmb
bpemb_en = BPEmb(lang="en",vs=50000)

In [44]:
data=[]
for i in df:
    
    for j in word_tokenize(i):
      temp=[]
      for sw in bpemb_en.encode(j):
        if sw[0]=='▁':sw=sw[1:]
        temp.append(sw.lower())
      data.append(temp)

In [8]:
data[:10]

[['good'],
 ['message'],
 ['for'],
 ['the'],
 ['society'],
 ['movie'],
 ['n', 'alli'],
 ['continuation'],
 ['ill', 'a'],
 ['nij', 'av', 'ag', 'alu']]

In [45]:
sw2v=gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 1)

In [46]:
import pickle
import numpy as np

embeddings_index = {}
mx=700
mx1=1000
words1=[]
for line in df:
    words = word_tokenize(line)
    for word in words:
        word=word.lower()
        words1.append(words)
        vec=[]
        vec.extend(w2v[word])
        for sw in bpemb_en.encode(word):
          if sw.startswith('▁'):sw=sw[1:]
          vec.extend(sw2v[sw.lower()])
        if len(vec)>mx : continue
        elif len(vec)<mx: vec.extend([0.0] * (mx - len(vec)))

        for char in word:
            vec.extend(c2v[char])
        if len(vec)>mx1 : continue
        elif len(vec)<mx1: vec.extend([0.0] * (mx1 - len(vec)))

        embeddings_index[word] =vec

In [ ]:
with open('ka_en_Skipgram.pkl', 'wb') as f:
	pickle.dump(embeddings_index, f)

In [ ]:
mx=len(embeddings_index['please'])
lll=[]
for word in list(embeddings_index.keys()):
    print(len(embeddings_index[word]))
    lll.append(len(embeddings_index[word]))
    

In [47]:
import pandas as pd
df =pd.read_csv('/content/drive/My Drive/knen/data/Dataset.csv',encoding='utf-8')


In [48]:
zerovec = [0.0] 
vecs=[]
for word in df.word:
    try:
        #vecs.append(wordvec[word.lower()])
        vecs.append(embeddings_index[word])
    except: vecs.append(zerovec*1000)

In [49]:
data_frame=pd.DataFrame(vecs)

data_frame1=data_frame.fillna(0.0)
#data_frame1

In [50]:
#spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_frame1, df.tag, test_size = 0.3)
print(X_train.shape)
print(X_test.shape)

(13602, 1000)
(5830, 1000)


In [51]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(penalty='l2',C=1.0)
lr.fit(X_train, y_train)

# Predicting the resultsax_iter=500,
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_pred_test,y_test))

              precision    recall  f1-score   support

          en       0.66      0.87      0.75      1391
       en-kn       0.02      0.73      0.04        11
          kn       0.93      0.60      0.73      4094
    location       0.19      0.90      0.32        10
        name       0.28      0.55      0.37       168
       other       0.14      0.50      0.22       156

    accuracy                           0.66      5830
   macro avg       0.37      0.69      0.40      5830
weighted avg       0.83      0.66      0.71      5830



In [60]:
from sklearn.svm import LinearSVC,SVC
lsvc=LinearSVC()
lsvc.fit(X_train, y_train)


LinearSVC()

In [61]:
y_pred_test = lsvc.predict(X_test)
print(classification_report(y_pred_test,y_test))

              precision    recall  f1-score   support

          en       0.66      0.86      0.75      1413
       en-kn       0.02      0.67      0.05        15
          kn       0.94      0.60      0.73      4121
    location       0.13      0.43      0.20        14
        name       0.25      0.54      0.34       152
       other       0.11      0.52      0.18       115

    accuracy                           0.66      5830
   macro avg       0.35      0.60      0.37      5830
weighted avg       0.83      0.66      0.71      5830



In [62]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
mlp.fit(X_train, y_train)
y_pred_test = mlp.predict(X_test)
print(classification_report(y_pred_test,y_test))

              precision    recall  f1-score   support

          en       0.65      0.87      0.74      1371
       en-kn       0.02      0.82      0.04        11
          kn       0.92      0.60      0.73      4045
    location       0.15      0.64      0.24        11
        name       0.32      0.50      0.39       210
       other       0.14      0.43      0.21       182

    accuracy                           0.66      5830
   macro avg       0.37      0.64      0.39      5830
weighted avg       0.81      0.66      0.70      5830



In [63]:
from sklearn.ensemble import VotingClassifier
lsvc=SVC(kernel='linear',probability=True)
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
lr=LogisticRegression()
estimators=[('lr', lr), ('lsvc', lsvc),('mlp', mlp)]
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('lsvc', SVC(kernel='linear', probability=True)),
                             ('mlp',
                              MLPClassifier(hidden_layer_sizes=(150, 100, 50),
                                            max_iter=300, random_state=1))],
                 voting='soft')

In [64]:
y_pred_test = ensemble.predict(X_test)
print(classification_report(y_pred_test,y_test))

              precision    recall  f1-score   support

          en       0.66      0.87      0.75      1385
       en-kn       0.02      0.77      0.05        13
          kn       0.94      0.60      0.74      4105
    location       0.13      0.86      0.22         7
        name       0.29      0.56      0.38       169
       other       0.14      0.50      0.22       151

    accuracy                           0.67      5830
   macro avg       0.36      0.69      0.39      5830
weighted avg       0.83      0.67      0.71      5830



n-grams


In [2]:
import pandas as pd
import re

In [3]:
df=pd.read_csv('/content/drive/My Drive/knen/data/Dataset.csv')


In [4]:
df.head()

,word,tag
0,alla,kn
1,kopista,kn
2,baruthe,kn
3,coolagiru,en-kn
4,madhuswamy,name


In [5]:
df.tag.value_counts()

kn          8720
en          6282
other       1763
en-kn       1472
name        1062
location     133
Name: tag, dtype: int64

In [6]:
from nltk import ngrams

In [7]:
def get_char_ngrams(word):
    n=4
    ngrams_list = []
    word = re.sub(r'ς', 'σ', word)
    ngrams_list.append(list(ngrams(word, n, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))
    
    # Removing redundant ngrams:
    if (n > 2):
        redundant_combinations = n - 2
        ngrams_list = [ngram_list[redundant_combinations : -redundant_combinations] for ngram_list in ngrams_list]
    
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = ''
    for i in range(0, n):
        format_string += ('%s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

def asciiPercentage(s):
	count = 0.
	for char in s:
		if ord(char) < 128:
			count += 1
	return count/len(s)

def vowelPercentage(s):
	vowels = "aeiou"
	count = 0.
	for char in s:
		if char in vowels:
			count += 1
	return count/len(s)

In [8]:
get_char_ngrams('university')

['_uni', 'univ', 'nive', 'iver', 'vers', 'ersi', 'rsit', 'sity', 'ity_']

In [18]:
def word2features(sent):

	# feature vector
	# word, pos, lang
    
    word = sent
    wordClean = ''.join([ch for ch in word if ch in 'asdfghjklqwertyuiopzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM']).lower()
    normalizedWord = wordClean.lower()
    
    anyCap = any(char.isupper() for char in word)
    allCap = all(char.isupper() for char in word)
    hasSpecial = any(ord(char) > 32 and ord(char) < 65 for char in word)
    
    
    hashTag = word[0] == '#'
    mention = word[0] == '@'
    
    
    '''features = {'word' : word, 'wordClean' : wordClean, 'normalizedWord' : normalizedWord, \
                'isTitle' : word.istitle(), 'wordLength' : len(word), \
                'anyCap' : anyCap, 'allCap' : word.isupper(),
                'hasSpecial' : hasSpecial, 'asciiPer' : asciiPercentage(word)}'''
    features = {'word' : word, 'wordClean' : wordClean, 'normalizedWord' : normalizedWord}
    
    
#     features['suffix5'] = word[-5:]
#     features['prefix5'] = word[:5]
#     features['suffix4'] = word[-4:]
#     features['prefix4'] = word[:4]
    features['suffix3'] = word[-3:]
    features['prefix3'] = word[:3]
    features['suffix2'] = word[-2:]
    features['prefix2'] = word[:2]
    features['suffix1'] = word[-1:]
    features['prefix1'] = word[:1]
    
    tmp=list(features.values())+get_char_ngrams(sent)
    for sw in bpemb_en.encode(sent):
      if sw.startswith('▁'):
        sw=sw[1:]
      tmp.extend(get_char_ngrams(sw)) 
    return tmp

In [20]:
from sklearn.model_selection import train_test_split

tr_df,te_df = train_test_split(df, test_size=0.3)


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer=word2features)

train_set_vectors = count_vect.fit_transform(tr_df.word)
test_set_vectors = count_vect.transform(te_df.word) # Unlike fit_transform(), transform() does not change the count vectorizer's vocabulary so it should be used for the test set.
train_set_vectors

<13602x37698 sparse matrix of type '<class 'numpy.int64'>'
	with 199924 stored elements in Compressed Sparse Row format>

In [31]:
from numpy import set_printoptions
import sys
set_printoptions(threshold=sys.maxsize) # Prints whole array. Required because by default an array with thousands of elements wouldn't be printed in full.
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [28]:
clf_multinomialNB = MultinomialNB() # There are no params for MultinomialDB that prevent overfitting, so any overfitting is caused by the small dataset size.
clf_multinomialNB.fit(train_set_vectors, tr_df.tag)

#test set
clf_multinomialNB_predictions = clf_multinomialNB.predict(test_set_vectors)

print('\t\t\tPERFORMANCE\n')
print('Accuracy:', round(accuracy_score(te_df.tag, clf_multinomialNB_predictions), 2), '\n')

print(classification_report(te_df.tag, clf_multinomialNB_predictions))

			PERFORMANCE

Accuracy: 0.78 

              precision    recall  f1-score   support

          en       0.86      0.82      0.84      1898
       en-kn       0.74      0.64      0.68       415
          kn       0.79      0.91      0.84      2638
    location       0.83      0.11      0.20        45
        name       0.50      0.42      0.45       313
       other       0.57      0.40      0.47       521

    accuracy                           0.78      5830
   macro avg       0.71      0.55      0.58      5830
weighted avg       0.77      0.78      0.77      5830



In [33]:
clf_linearSVC = LinearSVC(max_iter=1500) # n_samples < n_features in training set so the dual param is kept at its default value of True. Default max_iter = 1000
clf_linearSVC.fit(train_set_vectors, tr_df.tag)

clf_linearSVC_predictions = clf_linearSVC.predict(test_set_vectors)

print('\t\t\tPERFORMANCE\n')
print('Accuracy:', round(accuracy_score(te_df.tag, clf_linearSVC_predictions), 2), '\n')

print(classification_report(te_df.tag, clf_linearSVC_predictions))

			PERFORMANCE

Accuracy: 0.79 

              precision    recall  f1-score   support

          en       0.82      0.87      0.85      1898
       en-kn       0.86      0.63      0.73       415
          kn       0.81      0.90      0.85      2638
    location       0.71      0.22      0.34        45
        name       0.58      0.43      0.49       313
       other       0.60      0.38      0.46       521

    accuracy                           0.79      5830
   macro avg       0.73      0.57      0.62      5830
weighted avg       0.79      0.79      0.78      5830



In [29]:
clf_logisticRegression = LogisticRegression() # Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
clf_logisticRegression.fit(train_set_vectors, tr_df.tag)

clf_logisticRegression_predictions = clf_logisticRegression.predict(test_set_vectors)

print('\t\t\tPERFORMANCE\n')
print('Accuracy:', round(accuracy_score(te_df.tag, clf_logisticRegression_predictions), 2), '\n')

print(classification_report(te_df.tag, clf_logisticRegression_predictions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


			PERFORMANCE

Accuracy: 0.79 

              precision    recall  f1-score   support

          en       0.81      0.88      0.85      1898
       en-kn       0.91      0.61      0.73       415
          kn       0.80      0.91      0.85      2638
    location       0.70      0.16      0.25        45
        name       0.60      0.41      0.49       313
       other       0.63      0.33      0.44       521

    accuracy                           0.79      5830
   macro avg       0.74      0.55      0.60      5830
weighted avg       0.79      0.79      0.78      5830



In [30]:
from sklearn.neural_network import MLPClassifier

clf_knn = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
clf_knn.fit(train_set_vectors, tr_df.tag)

clf_knn_predictions = clf_knn.predict(test_set_vectors)

print('\t\t\tPERFORMANCE\n')
print('Accuracy:', round(accuracy_score(te_df.tag, clf_knn_predictions), 2), '\n')

print(classification_report(te_df.tag, clf_knn_predictions))

			PERFORMANCE

Accuracy: 0.79 

              precision    recall  f1-score   support

          en       0.83      0.85      0.84      1898
       en-kn       0.83      0.64      0.72       415
          kn       0.83      0.87      0.85      2638
    location       0.69      0.24      0.36        45
        name       0.52      0.48      0.50       313
       other       0.50      0.50      0.50       521

    accuracy                           0.79      5830
   macro avg       0.70      0.60      0.63      5830
weighted avg       0.78      0.79      0.78      5830



In [37]:
from sklearn.ensemble import VotingClassifier
lsvc=SVC(kernel='linear',probability=True)
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
lr=LogisticRegression()
estimators=[('lr', lr), ('lsvc', lsvc),('mlp', mlp)]
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(train_set_vectors, tr_df.tag)

y_pred = ensemble.predict(test_set_vectors)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [38]:
print(classification_report(te_df.tag, y_pred))

              precision    recall  f1-score   support

          en       0.83      0.87      0.85      1898
       en-kn       0.87      0.66      0.75       415
          kn       0.82      0.89      0.85      2638
    location       0.75      0.27      0.39        45
        name       0.56      0.45      0.50       313
       other       0.57      0.44      0.50       521

    accuracy                           0.80      5830
   macro avg       0.73      0.60      0.64      5830
weighted avg       0.79      0.80      0.79      5830

